# kgl

> Command line utilities for kaggle competitions

In [ ]:
#| default_exp kgl

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.all import *
from fastkaggle import *
import itertools as it

from oztools.core import *
from oztools.nbd import *

This module requires kaggle API token in order to work.
See [here](https://github.com/Kaggle/kaggle-api/blob/main/docs/README.md#api-credentials) for info
on how to setup that.

## Competition utils

Modified version of `setup_comp` from `fastkaggle`.
I like to put my data into `data` folders so it's easier to mask them in version control.

In [ ]:
#|export
def setup_comp(competition, install=''):
    "Get a path to data for `competition`, downloading it if needed"
    if iskaggle:
        if install:
            os.system(f'pip install -Uqq {install}')
        return Path('../input')/competition
    else:
        path = Path('./var')/competition
        api = import_kaggle()
        if not path.exists():
            import zipfile
            api.competition_download_cli(str(competition), path='./var')
            zipfile.ZipFile(f'./var/{competition}.zip').extractall(path)
        return path

## Setup competition projects

In [ ]:
api = import_kaggle()
comps = api.competitions_list()
comp = comps[0]
comp.title, comp.url.split("/")[-1]

('AI Mathematical Olympiad - Progress Prize 2',
 'ai-mathematical-olympiad-progress-prize-2')

In [ ]:
len("equity-post-HCT-survival-prediction  ")

37

In [ ]:
#| export
def disp_comp(comp):
    slug = comp.url.split("/")[-1]
    return f"{pad(slug[:40],40)} {comp.title[:40]}"

In [ ]:
joinedkey = attrkey("userHasEntered")

In [ ]:
comps.sort(key=joinedkey)
active, entered = (list(y) for x,y in it.groupby(comps, lambda x: x.userHasEntered))

In [ ]:
#| export
def get_competitions():
    api = import_kaggle()
    comps = api.competitions_list()
    
    joinedkey = attrkey("userHasEntered")
    comps.sort(key=joinedkey)
    active, entered = (list(y) for x,y in it.groupby(comps, joinedkey))
    return active, entered

In [ ]:
get_competitions()

([https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2,
  https://www.kaggle.com/competitions/konwinski-prize,
  https://www.kaggle.com/competitions/llms-you-cant-please-them-all,
  https://www.kaggle.com/competitions/fide-google-efficiency-chess-ai-challenge,
  https://www.kaggle.com/competitions/lux-ai-season-3,
  https://www.kaggle.com/competitions/titanic,
  https://www.kaggle.com/competitions/home-data-for-ml-course,
  https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques,
  https://www.kaggle.com/competitions/spaceship-titanic,
  https://www.kaggle.com/competitions/digit-recognizer,
  https://www.kaggle.com/competitions/nlp-getting-started,
  https://www.kaggle.com/competitions/connectx,
  https://www.kaggle.com/competitions/gan-getting-started,
  https://www.kaggle.com/competitions/llm-classification-finetuning,
  https://www.kaggle.com/competitions/tpu-getting-started,
  https://www.kaggle.com/competitions/contradictory-my-dear

In [ ]:
#| export
@call_parse
def kgl_list():
    "List kaggle competitions"

    active, entered = get_competitions()

    return '\n'.join(("Joined:", *str_enumerate(map(disp_comp, entered), 1),
                      "Active:", *str_enumerate(map(disp_comp, active), 1+len(entered))))

In [ ]:
print(kgl_list())

Joined:
  1   equity-post-HCT-survival-predictions     CIBMTR - Equity in post-HCT Survival Pre
  2   playground-series-s5e2                   Backpack Prediction Challenge
  3   store-sales-time-series-forecasting      Store Sales - Time Series Forecasting
  4   jane-street-real-time-market-data-foreca Jane Street Real-Time Market Data Foreca
Active:
  5   ai-mathematical-olympiad-progress-prize- AI Mathematical Olympiad - Progress Priz
  6   konwinski-prize                          Konwinski Prize
  7   llms-you-cant-please-them-all            LLMs - You Can't Please Them All
  8   fide-google-efficiency-chess-ai-challeng FIDE & Google Efficient Chess AI Challen
  9   lux-ai-season-3                          NeurIPS 2024 - Lux AI Season 3
  10  titanic                                  Titanic - Machine Learning from Disaster
  11  home-data-for-ml-course                  Housing Prices Competition for Kaggle Le
  12  house-prices-advanced-regression-techniq House Prices - Advanced Re

In [ ]:
#| export
def maybe_int(x: str):
    try:
        return int(x)
    except ValueError:
        return x

In [ ]:
comp = comps[0]

In [ ]:
comp.url

'https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2'

In [ ]:
comp.title

'AI Mathematical Olympiad - Progress Prize 2'

In [ ]:
#| export
def get_competition(n: str):
    active, entered = get_competitions()
    comps = entered + active

    try:
        try: return comps[int(n)-1]
        except ValueError: return L(comps).filter(lambda x: n in x.url.split("/")[-1])[0]
    except IndexError:
        warn("Couldn't find competition")
        return

In [ ]:
#| export
@call_parse
def kgl_new(n: str, # competition id or name
            save_to: str # project name to use locally and for github
            ):
    "Setup nbdev environment for a kaggle competition"
    comp = get_competition(n)
    if not comp:
        return

    save_to = "kaggle_" + save_to
    
    print(f'Loading competition "{comp.title}" into "{save_to}"')
    nbd_new_fn(save_to, f'Code for [{comp.title}]({comp.url}) Kaggle competition')

[link](example.com)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()